In [13]:
from SBM_SDE import *
from obs_and_flow_classes_and_functions import *
from get_CO2 import *
import seaborn as sns
import torch
from torch import nn
import torch.distributions as d
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [14]:
torch.manual_seed(2)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = .2 #SDE discretization timestep.
t = 150 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
l_r = 1e-3
niter = 1500
piter = 100
batch_size = 2 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.

In [15]:
obs_times, obs_means_CON, obs_error_CON = csv_to_obs_df('CON_synthetic_sol_df.csv', state_dim_SCON + 1, t, 0.1) #Need to +1 because data has CO2 observations on top of other state observations.
print(obs_means_CON)
print(obs_error_CON)

tensor([[4.5660e+01, 4.5647e+01, 4.5650e+01, 4.5643e+01, 4.5638e+01, 4.5640e+01,
         4.5626e+01, 4.5630e+01, 4.5618e+01, 4.5617e+01, 4.5615e+01, 4.5605e+01,
         4.5609e+01, 4.5595e+01, 4.5598e+01, 4.5591e+01],
        [7.1469e-02, 7.3700e-02, 7.3289e-02, 7.4423e-02, 7.5230e-02, 7.4875e-02,
         7.6935e-02, 7.6316e-02, 7.8034e-02, 7.7924e-02, 7.8087e-02, 7.9339e-02,
         7.8534e-02, 8.0440e-02, 7.9792e-02, 8.0558e-02],
        [7.1469e-01, 7.1372e-01, 7.1409e-01, 7.1363e-01, 7.1332e-01, 7.1362e-01,
         7.1266e-01, 7.1317e-01, 7.1240e-01, 7.1253e-01, 7.1251e-01, 7.1189e-01,
         7.1246e-01, 7.1149e-01, 7.1193e-01, 7.1158e-01],
        [1.1000e-03, 1.8381e-03, 4.6086e-04, 3.0443e-03, 4.7108e-04, 1.8966e-03,
         1.1670e-03, 7.0596e-04, 2.7906e-03, 4.2941e-04, 2.8298e-03, 7.3398e-04,
         1.2239e-03, 2.0338e-03, 5.1562e-04, 3.3495e-03]])
tensor([[4.5624e+00, 7.6809e-03, 7.1285e-02, 1.5369e-04]])


In [16]:
obs_times, obs_means_AWB, obs_error_AWB = csv_to_obs_df('AWB_synthetic_sol_df.csv', state_dim_SAWB + 1, t, 0.1) #Need to +1 because data has CO2 observations on top of other state observations.
print(obs_means_AWB)
print(obs_error_AWB)

tensor([[5.3602e+01, 5.3588e+01, 5.3591e+01, 5.3584e+01, 5.3578e+01, 5.3579e+01,
         5.3564e+01, 5.3569e+01, 5.3560e+01, 5.3559e+01, 5.3558e+01, 5.3545e+01,
         5.3551e+01, 5.3536e+01, 5.3538e+01, 5.3531e+01],
        [1.1421e-01, 1.1832e-01, 1.1810e-01, 1.1999e-01, 1.2125e-01, 1.2086e-01,
         1.2401e-01, 1.2294e-01, 1.2498e-01, 1.2491e-01, 1.2482e-01, 1.2692e-01,
         1.2544e-01, 1.2826e-01, 1.2713e-01, 1.2800e-01],
        [6.7073e-01, 6.7239e-01, 6.7187e-01, 6.7274e-01, 6.7348e-01, 6.7354e-01,
         6.7549e-01, 6.7470e-01, 6.7601e-01, 6.7623e-01, 6.7639e-01, 6.7818e-01,
         6.7750e-01, 6.7964e-01, 6.7946e-01, 6.8049e-01],
        [1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3416e-02,
         1.3416e-02, 1.3416e-02, 1.3417e-02, 1.3417e-02, 1.3418e-02, 1.3419e-02,
         1.3419e-02, 1.3420e-02, 1.3421e-02, 1.3422e-02],
        [1.1000e-03, 1.6757e-03, 5.7473e-04, 2.4797e-03, 5.9680e-04, 1.7478e-03,
         1.2328e-03, 8.2910e-04, 2.4061

In [17]:
obs_times, obs_means_AWB_ECA, obs_error_AWB_ECA = csv_to_obs_df('AWB_ECA_synthetic_sol_df.csv', state_dim_SAWB + 1, t, 0.1) #Need to +1 because data has CO2 observations on top of other state observations.
print(obs_means_AWB_ECA)
print(obs_error_AWB_ECA)

tensor([[5.3606e+01, 5.3592e+01, 5.3594e+01, 5.3586e+01, 5.3579e+01, 5.3581e+01,
         5.3567e+01, 5.3572e+01, 5.3559e+01, 5.3558e+01, 5.3555e+01, 5.3545e+01,
         5.3550e+01, 5.3533e+01, 5.3538e+01, 5.3528e+01],
        [1.9081e-01, 1.9506e-01, 1.9527e-01, 1.9779e-01, 1.9962e-01, 1.9968e-01,
         2.0286e-01, 2.0196e-01, 2.0493e-01, 2.0501e-01, 2.0569e-01, 2.0746e-01,
         2.0642e-01, 2.0993e-01, 2.0873e-01, 2.1039e-01],
        [6.7073e-01, 6.7236e-01, 6.7196e-01, 6.7284e-01, 6.7358e-01, 6.7335e-01,
         6.7505e-01, 6.7448e-01, 6.7616e-01, 6.7630e-01, 6.7668e-01, 6.7806e-01,
         6.7749e-01, 6.7978e-01, 6.7921e-01, 6.8067e-01],
        [1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3416e-02,
         1.3416e-02, 1.3416e-02, 1.3417e-02, 1.3417e-02, 1.3418e-02, 1.3418e-02,
         1.3419e-02, 1.3420e-02, 1.3421e-02, 1.3422e-02],
        [1.1000e-03, 1.6543e-03, 5.6910e-04, 2.4475e-03, 5.8824e-04, 1.7276e-03,
         1.2063e-03, 8.1481e-04, 2.3599

In [18]:
temp_ref = 283

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}
SCON_SS_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC}

In [19]:
#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [20]:
#System parameters from deterministic model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_ECA_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_ECA_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [21]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [22]:
def neg_log_lik(C_PATH, T_SPAN_TENSOR, DT, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF):
    drift, diffusion_sqrt = DRIFT_DIFFUSION(C_PATH[:, :-1, :], T_SPAN_TENSOR[:, :-1, :], I_S_TENSOR[:, :-1, :], I_D_TENSOR[:, :-1, :], PARAMS_DICT, TEMP_GEN, TEMP_REF)
    euler_maruyama_state_sample_object = d.multivariate_normal.MultivariateNormal(loc = C_PATH[:, :-1, :] + drift * DT, scale_tril = diffusion_sqrt * math.sqrt(DT))
    return -euler_maruyama_state_sample_object.log_prob(C_PATH[:, 1:, :]).sum(-1)

In [23]:
def train(DEVICE, L_R, NITER, PRETRAIN_ITER, BATCH_SIZE, ObsModel, csv_to_obs_df, DATA_CSV, OBS_ERROR_SCALE, STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF, ANALYTICAL_STEADY_STATE_INIT, GET_CO2):
    obs_times, obs_means, obs_error = csv_to_obs_df(DATA_CSV, STATE_DIM + 1, T, OBS_ERROR_SCALE) #Need to +1 because data has CO2 observations on top of other state observations. 
    obs_model_no_CO2 = ObsModel(DEVICE, obs_times, DT, obs_means[:-1, :], obs_error[:, :-1]) #Hack for bypassing ObsModel and SDEFlow dimension mismatch issue.
    obs_model_CO2 = ObsModel(DEVICE, obs_times, DT, obs_means, obs_error)
    net = SDEFlow(DEVICE, BATCH_SIZE, obs_model_no_CO2, STATE_DIM, T, DT, N).to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr = L_R) 
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    C0 = ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), PARAMS_DICT) #Calculate deterministic initial conditions.
    C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Assign initial conditions to C_PATH.
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for iter in range(NITER):
            net.train()
            optimizer.zero_grad()
            C_PATH, log_prob = net() #Obtain paths with solutions at times after t0.
            C_PATH = torch.cat([C0, C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            CO2 = GET_CO2(C_PATH, T_SPAN_TENSOR, PARAMS_DICT, TEMP_GEN, TEMP_REF)
            x_with_CO2 = torch.cat([C_PATH, CO2], -1)
            if iter <= PRETRAIN_ITER:
                #l1_norm_element = x_with_CO2 - torch.mean(obs_model_CO2.mu, -1)
                #l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                #best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                #norm_losses.append(l1_norm.item())
                l2_norm_element = x_with_CO2 - torch.mean(obs_model_CO2.mu, -1)
                l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l2_norm.item())
                if len(norm_losses) > 10:
                    norm_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average norm loss at {iter} iterations is: {sum(norm_losses) / len(norm_losses)}. Best norm loss value is: {best_loss_norm}.")
                    print('\nx with CO2 means across time =', x_with_CO2.mean(-2))
                    print('\nx with CO2 =', x_with_CO2)
                #l1_norm.backward()
                l2_norm.backward()
            else:
                log_lik = neg_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), dt, I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE), DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF)
                ELBO = log_prob.mean() + log_lik.mean() - obs_model_CO2(x_with_CO2) #obs_model_CO2(x_with_CO2) is obs log likelihood.
                best_loss_ELBO = ELBO if ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(ELBO.item())
                if len(ELBO_losses) > 10:
                    ELBO_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average ELBO loss at {iter} iterations is: {sum(ELBO_losses) / len(ELBO_losses)}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nx with CO2 means across time =', x_with_CO2.mean(-2))
                    print('\nx with CO2 =', x_with_CO2)
                ELBO.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if iter % 100000 == 0 and iter > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            tq.update()

In [24]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON, get_CO2_CON)


Train Diffusion:   0%|          | 0/1500 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000000173.503454. Best norm loss value is: 1735.0345458984375.

x with CO2 means across time = tensor([[0.9018, 0.7048, 0.6691, 0.0029],
        [0.8806, 0.7082, 0.6642, 0.0030]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [7.0500e-01, 9.6841e-01, 6.6545e-01, 3.4796e-03],
         [6.3360e-01, 5.9831e-01, 6.7865e-01, 2.2781e-03],
         ...,
         [6.9264e-01, 6.2056e-01, 5.5705e-01, 4.8133e-03],
         [6.9130e-01, 5.4380e-01, 6.3485e-01, 4.2789e-03],
         [1.0544e+00, 9.8098e-01, 8.4000e-01, 7.6251e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [7.0500e-01, 7.1802e-01, 5.3653e-01, 2.5892e-03],
         [7.3629e-01, 9.0377e-01, 8.6091e-01, 3.4132e-03],
         ...,
         [2.2301e+00, 8.0130e-01, 4.7395e-01, 6.2161e-03],
         [7.8987e-01, 7.1217e-01, 8.7588e-01, 5.6105e-03],
         [4.5167e-01, 7.3495e-01, 3.9585e-01, 5.6243e-03


Train Diffusion:   1%|          | 10/1500 [00:06<14:56,  1.66it/s]

Moving average norm loss at 10 iterations is: 1638.4563354492188. Best norm loss value is: 1579.98974609375.

x with CO2 means across time = tensor([[5.1876e+00, 4.0139e-01, 1.7050e-01, 1.7593e-03],
        [5.0817e+00, 3.9070e-01, 1.7059e-01, 1.7160e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [1.1752e+00, 5.1720e+00, 2.5150e-01, 1.8077e-02],
         [6.5294e-01, 2.4075e+00, 2.1283e-01, 8.7660e-03],
         ...,
         [1.0348e+01, 2.4089e-01, 1.3883e-01, 2.4028e-03],
         [8.8505e-01, 2.1575e-01, 1.0223e-01, 1.6869e-03],
         [7.3108e+00, 3.1347e-01, 2.1101e-01, 2.8075e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [8.7575e-01, 4.8737e-01, 2.5008e-01, 1.7486e-03],
         [4.0852e+00, 2.4645e-01, 2.9152e-01, 1.0122e-03],
         ...,
         [3.3662e-01, 2.5399e-01, 8.0090e-02, 1.9290e-03],
         [9.1428e+00, 3.1369e-01, 1.3354e-01, 2.8846e-03],
         [4.3999e


Train Diffusion:   1%|▏         | 20/1500 [00:11<13:36,  1.81it/s]

...


Train Diffusion:  99%|█████████▊| 1480/1500 [14:40<00:11,  1.78it/s]

Moving average ELBO loss at 1480 iterations is: -8576.7474609375. Best ELBO loss value is: -9223.109375.

x with CO2 means across time = tensor([[3.2432e+01, 1.5805e-01, 7.2767e-01, 1.5252e-03],
        [3.2203e+01, 1.4933e-01, 7.0589e-01, 1.4939e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.5967e+01, 1.2535e-02, 1.0665e-01, 8.7529e-04],
         [4.6167e+01, 1.9983e-05, 8.8279e-02, 8.8269e-04],
         ...,
         [3.3099e+01, 8.6009e-02, 8.4544e-01, 2.7205e-03],
         [3.2864e+01, 8.5502e-02, 6.8262e-01, 2.6660e-03],
         [3.3593e+01, 1.1031e-01, 7.1337e-01, 2.9045e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.5234e+01, 6.3340e-02, 2.2813e-01, 1.0562e-03],
         [4.4862e+01, 8.9635e-02, 3.6158e-01, 1.2224e-03],
         ...,
         [3.3496e+01, 1.8225e-01, 7.1572e-01, 3.4188e-03],
         [3.4347e+01, 1.5014e-01, 8.3462e-01, 3.2751e-03],
         [3.4121e+01,


Train Diffusion:  99%|█████████▉| 1490/1500 [14:46<00:05,  1.79it/s]

Moving average ELBO loss at 1490 iterations is: -8782.5734375. Best ELBO loss value is: -9223.109375.

x with CO2 means across time = tensor([[3.2487e+01, 1.5609e-01, 7.2238e-01, 1.5417e-03],
        [3.2502e+01, 1.3470e-01, 6.3830e-01, 1.4097e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.4723e+01, 4.2487e-02, 2.3029e-01, 9.7478e-04],
         [4.4847e+01, 3.1433e-05, 2.1210e-01, 8.7580e-04],
         ...,
         [3.3136e+01, 7.1857e-02, 4.8399e-01, 2.5101e-03],
         [3.3230e+01, 7.1167e-02, 4.6277e-01, 2.5140e-03],
         [3.2945e+01, 6.8319e-02, 5.8161e-01, 2.5159e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.5543e+01, 4.5851e-02, 9.0858e-02, 9.8164e-04],
         [4.4525e+01, 5.7471e-02, 1.5890e-01, 1.0701e-03],
         ...,
         [3.4057e+01, 1.6685e-01, 1.0074e+00, 3.4221e-03],
         [3.4524e+01, 1.5315e-01, 9.5647e-01, 3.3437e-03],
         [3.5209e+01, 1.


Train Diffusion: 100%|██████████| 1500/1500 [14:52<00:00,  1.68it/s]


In [25]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_SS, SCON_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON, get_CO2_CON)


Train Diffusion:   0%|          | 0/1500 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000000173.462536. Best norm loss value is: 1734.6253662109375.

x with CO2 means across time = tensor([[0.8956, 0.7651, 0.8761, 0.0032],
        [0.9122, 0.7724, 0.8658, 0.0032]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [6.3355e-01, 1.6942e-01, 7.9433e-01, 7.1230e-04],
         [5.2189e-01, 5.2951e-01, 1.2397e+00, 2.1081e-03],
         ...,
         [1.1305e+00, 8.0522e-01, 1.0281e+00, 6.3491e-03],
         [1.4903e+00, 6.9828e-01, 1.0500e+00, 5.5978e-03],
         [3.2666e-01, 1.0210e+00, 1.2060e+00, 7.9920e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [8.0993e-01, 8.1435e-01, 7.1232e-01, 2.9512e-03],
         [5.4826e-01, 6.7620e-01, 7.0966e-01, 2.5632e-03],
         ...,
         [7.8991e-01, 7.7274e-01, 1.0775e+00, 6.1037e-03],
         [5.4715e-01, 9.5231e-01, 6.1170e-01, 7.3073e-03],
         [1.0332e+00, 6.2559e-01, 1.2953e+00, 5.1093e-03


Train Diffusion:   1%|          | 10/1500 [00:05<13:46,  1.80it/s]

Moving average norm loss at 10 iterations is: 1651.3329345703125. Best norm loss value is: 1594.8743896484375.

x with CO2 means across time = tensor([[4.8028e+00, 4.3586e-01, 2.5472e-01, 1.9042e-03],
        [4.6365e+00, 4.4959e-01, 2.5624e-01, 1.9569e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [2.2560e+00, 3.3534e-01, 3.3035e-01, 1.2546e-03],
         [1.7110e-01, 3.2339e-01, 3.4310e-01, 1.2247e-03],
         ...,
         [1.7500e-01, 3.4067e-01, 2.1813e-01, 2.6049e-03],
         [7.1430e-01, 3.6872e-01, 1.8693e-01, 2.8423e-03],
         [2.0078e-01, 2.8989e-01, 2.6824e-01, 2.2531e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.9461e-01, 6.3036e+00, 3.4465e-01, 2.2021e-02],
         [6.4205e+00, 4.1208e+00, 2.3769e-01, 1.5086e-02],
         ...,
         [5.3455e+00, 3.0853e-01, 1.7550e-01, 2.6395e-03],
         [1.0092e+01, 3.7863e-01, 2.2925e-01, 3.4497e-03],
         [5.051


Train Diffusion:   1%|▏         | 20/1500 [00:11<13:57,  1.77it/s]

...


Train Diffusion:  99%|█████████▊| 1480/1500 [16:14<00:11,  1.81it/s]

Moving average ELBO loss at 1480 iterations is: -9048.13271484375. Best ELBO loss value is: -9274.7958984375.

x with CO2 means across time = tensor([[3.7653e+01, 1.4382e-01, 6.5423e-01, 1.5887e-03],
        [3.7722e+01, 1.3289e-01, 6.6069e-01, 1.5418e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.4006e+01, 8.6820e-02, 2.2157e-01, 1.1153e-03],
         [4.2403e+01, 8.8137e-02, 3.3056e-01, 1.1662e-03],
         ...,
         [3.9006e+01, 1.5546e-01, 4.8817e-01, 3.4569e-03],
         [3.9076e+01, 1.4866e-01, 5.0684e-01, 3.4291e-03],
         [4.0677e+01, 9.7035e-02, 5.7789e-01, 3.1592e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.5787e+01, 2.5457e-02, 1.8396e-01, 9.2789e-04],
         [4.6379e+01, 1.6270e-02, 1.6558e-01, 9.5676e-04],
         ...,
         [3.7943e+01, 7.8132e-02, 1.0459e+00, 2.9895e-03],
         [3.9170e+01, 6.3550e-02, 9.8805e-01, 2.9437e-03],
         [3.9162


Train Diffusion:  99%|█████████▉| 1490/1500 [16:20<00:05,  1.78it/s]

Moving average ELBO loss at 1490 iterations is: -8828.1890625. Best ELBO loss value is: -9274.7958984375.

x with CO2 means across time = tensor([[3.7809e+01, 1.3182e-01, 6.8682e-01, 1.5441e-03],
        [3.7670e+01, 1.4342e-01, 6.5939e-01, 1.5813e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.4301e+01, 2.8840e-02, 1.5676e-01, 9.0947e-04],
         [4.2518e+01, 1.9800e-02, 2.6536e-01, 9.1126e-04],
         ...,
         [3.8443e+01, 7.9408e-02, 7.2898e-01, 2.9324e-03],
         [3.9764e+01, 6.1362e-02, 6.3322e-01, 2.8547e-03],
         [3.9483e+01, 6.2172e-02, 6.9084e-01, 2.8664e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.6193e+01, 5.3662e-02, 2.2715e-01, 1.0394e-03],
         [4.6537e+01, 1.1593e-01, 2.0851e-01, 1.3271e-03],
         ...,
         [3.8438e+01, 1.1421e-01, 8.2510e-01, 3.2191e-03],
         [3.8358e+01, 1.1824e-01, 8.5840e-01, 3.2674e-03],
         [4.0203e+01


Train Diffusion: 100%|██████████| 1500/1500 [16:25<00:00,  1.52it/s]


In [26]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'AWB_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_C, SAWB_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB, get_CO2_AWB)


Train Diffusion:   0%|          | 0/1500 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000000204.632055. Best norm loss value is: 2046.320556640625.

x with CO2 means across time = tensor([[0.8212, 0.7313, 0.7281, 0.7032, 0.0038],
        [0.8055, 0.7174, 0.7358, 0.7011, 0.0039]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [5.0048e-01, 7.8354e-01, 5.1454e-01, 3.1434e-01, 2.9234e-03],
         [1.6978e+00, 5.1193e-01, 1.1579e+00, 8.5772e-01, 3.8522e-03],
         ...,
         [7.3163e-01, 7.9650e-01, 6.7072e-01, 6.7377e-01, 7.8933e-03],
         [2.5567e-01, 7.6441e-01, 1.0203e+00, 1.3821e+00, 1.0241e-02],
         [2.0647e-01, 8.9333e-01, 8.1098e-01, 2.4515e+00, 9.8064e-03]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [5.0944e-01, 2.0780e-01, 8.0305e-01, 1.0342e+00, 1.3829e-03],
         [2.4315e-01, 3.4541e-01, 5.9183e-01, 6.0675e-01, 1.8309e-03],
         ...,
         [4.5323e-01, 2.7007e-01, 9.3526e-01, 1.194


Train Diffusion:   1%|          | 10/1500 [00:10<23:45,  1.04it/s]

Moving average norm loss at 10 iterations is: 1910.0333374023437. Best norm loss value is: 1840.87646484375.

x with CO2 means across time = tensor([[6.7496e+00, 3.3435e-01, 2.9968e-01, 2.6832e-01, 1.1678e-03],
        [6.3553e+00, 3.2785e-01, 2.9968e-01, 2.6901e-01, 1.1609e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [3.6776e-01, 4.3144e+00, 3.3762e-01, 3.1233e-01, 4.2945e-03],
         [4.6562e-02, 1.2812e+00, 3.0955e-01, 2.4799e-01, 2.6561e-03],
         ...,
         [5.7925e-01, 3.0573e-01, 2.8284e-01, 2.5052e-01, 1.9843e-03],
         [7.9920e+00, 3.3324e-01, 3.3857e-01, 2.4196e-01, 2.4677e-03],
         [5.8604e+00, 2.9927e-01, 2.3647e-01, 3.3287e-01, 1.6868e-03]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [1.0569e+00, 7.2549e-01, 2.9944e-01, 2.4749e-01, 1.7878e-03],
         [5.0500e+00, 4.9343e-01, 2.8402e-01, 3.1342e-01, 1.3680e-03],
         ...,
   


Train Diffusion:   1%|▏         | 20/1500 [00:18<20:44,  1.19it/s]

...


Train Diffusion:  99%|█████████▊| 1480/1500 [20:56<00:17,  1.15it/s]

Moving average ELBO loss at 1480 iterations is: -10746.69521484375. Best ELBO loss value is: -11444.6513671875.

x with CO2 means across time = tensor([[3.1430e+01, 1.9740e-01, 5.6867e-01, 3.8213e-02, 1.2150e-03],
        [3.1596e+01, 1.8680e-01, 5.8830e-01, 3.5468e-02, 1.1366e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [5.2705e+01, 2.6597e-01, 1.8971e-01, 3.8670e-02, 5.7761e-04],
         [5.3252e+01, 2.0608e-01, 2.8173e-01, 1.9440e-02, 6.7707e-04],
         ...,
         [2.9048e+01, 1.1335e-01, 6.6428e-01, 3.4015e-02, 1.5441e-03],
         [2.9127e+01, 1.2709e-01, 8.4192e-01, 1.8634e-02, 1.9870e-03],
         [2.9760e+01, 1.6710e-01, 8.1010e-01, 1.3000e-02, 2.5061e-03]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [5.3274e+01, 4.8902e-02, 2.8751e-01, 6.0442e-02, 1.7531e-04],
         [5.2435e+01, 6.3111e-02, 3.6718e-01, 3.5305e-02, 2.8110e-04],
         ...,



Train Diffusion:  99%|█████████▉| 1490/1500 [21:06<00:09,  1.04it/s]

Moving average ELBO loss at 1490 iterations is: -10571.54697265625. Best ELBO loss value is: -11444.6513671875.

x with CO2 means across time = tensor([[3.1438e+01, 1.9430e-01, 5.7977e-01, 3.7480e-02, 1.1837e-03],
        [3.1461e+01, 1.9527e-01, 5.9385e-01, 4.5645e-02, 1.2230e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [5.3975e+01, 2.6244e-02, 1.9005e-01, 4.0330e-02, 6.8332e-05],
         [5.2856e+01, 5.3372e-02, 1.1777e-01, 2.0279e-02, 9.3125e-05],
         ...,
         [2.7966e+01, 9.0792e-02, 6.5611e-01, 2.1924e-02, 1.2431e-03],
         [2.7824e+01, 1.5789e-01, 8.1754e-01, 2.8550e-02, 2.3892e-03],
         [2.7275e+01, 1.8864e-01, 8.2165e-01, 1.1765e-02, 2.8223e-03]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [5.3496e+01, 8.0736e-02, 3.3627e-01, 6.4817e-02, 3.1927e-04],
         [5.3401e+01, 1.5172e-01, 5.4587e-01, 3.8164e-02, 8.4648e-04],
         ...,



Train Diffusion: 100%|██████████| 1500/1500 [21:15<00:00,  1.18it/s]


In [27]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'AWB_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_SS, SAWB_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB, get_CO2_AWB)


Train Diffusion:   0%|          | 0/1500 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000000203.98761. Best norm loss value is: 2039.8760986328125.

x with CO2 means across time = tensor([[0.9930, 0.7703, 0.8116, 0.9590, 0.0043],
        [0.9713, 0.7657, 0.7944, 0.9441, 0.0042]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [6.4775e-01, 3.4287e-01, 8.6041e-01, 6.2801e-01, 2.2312e-03],
         [1.5732e-01, 6.4508e-01, 1.7085e+00, 1.1807e+00, 5.7022e-03],
         ...,
         [1.2824e+00, 4.4171e-01, 8.4162e-01, 1.2103e+00, 5.9351e-03],
         [1.4428e+00, 6.3976e-01, 1.0650e+00, 1.4251e+00, 9.2108e-03],
         [1.7944e+00, 6.8930e-01, 1.2012e+00, 6.6177e-01, 1.0485e-02]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [6.4775e-01, 1.6459e+00, 4.4656e-01, 1.2297e+00, 3.9605e-03],
         [1.5587e+00, 8.9412e-01, 3.2431e-01, 4.5500e-01, 2.2679e-03],
         ...,
         [7.6642e-01, 6.4370e-01, 1.3210e+00, 5.516


Train Diffusion:   1%|          | 10/1500 [00:08<22:40,  1.10it/s]

Moving average norm loss at 10 iterations is: 1917.5589233398437. Best norm loss value is: 1842.01220703125.

x with CO2 means across time = tensor([[6.4150e+00, 3.3958e-01, 3.2236e-01, 3.0478e-01, 1.2335e-03],
        [6.2538e+00, 3.3557e-01, 3.2052e-01, 3.0450e-01, 1.2290e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [7.9929e-01, 7.1211e+00, 3.1361e-01, 2.7508e-01, 4.4120e-03],
         [7.1906e-01, 6.2322e-01, 2.3366e-01, 2.5978e-01, 1.3607e-03],
         ...,
         [1.2916e+01, 3.8865e-01, 3.3224e-01, 3.0672e-01, 2.7353e-03],
         [9.4408e-02, 2.3560e-01, 4.0863e-01, 2.1529e-01, 2.1410e-03],
         [2.5924e-01, 4.2613e-01, 3.0672e-01, 4.0000e-01, 2.7610e-03]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [7.9930e-01, 1.2974e-01, 3.5000e-01, 2.9649e-01, 5.1136e-04],
         [3.0136e+00, 3.6898e+00, 4.4166e-01, 3.1654e-01, 5.5102e-03],
         ...,
   


Train Diffusion:   1%|▏         | 20/1500 [00:16<19:00,  1.30it/s]

...


Train Diffusion:  99%|█████████▊| 1480/1500 [19:47<00:18,  1.10it/s]

Moving average ELBO loss at 1480 iterations is: -9364.54208984375. Best ELBO loss value is: -10534.466796875.

x with CO2 means across time = tensor([[3.5569e+01, 2.0906e-01, 7.4083e-01, 6.2404e-02, 1.4697e-03],
        [3.5860e+01, 2.0292e-01, 6.9961e-01, 5.6571e-02, 1.4588e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [5.3262e+01, 2.1146e-02, 4.0258e-01, 6.8168e-02, 9.9640e-05],
         [5.1135e+01, 1.5337e-02, 4.0841e-01, 1.3860e-02, 7.6333e-05],
         ...,
         [3.5231e+01, 8.4313e-02, 6.2121e-01, 9.5095e-03, 1.1195e-03],
         [3.4252e+01, 8.9647e-02, 7.7873e-01, 2.4897e-02, 1.3662e-03],
         [3.4004e+01, 1.9379e-01, 4.8122e-01, 1.6668e-02, 2.0380e-03]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [5.0973e+01, 1.5707e-01, 5.4334e-01, 8.0032e-02, 8.3636e-04],
         [5.1673e+01, 1.8457e-01, 7.5555e-01, 6.7852e-02, 1.2471e-03],
         ...,
  


Train Diffusion:  99%|█████████▉| 1490/1500 [19:55<00:08,  1.24it/s]

Moving average ELBO loss at 1490 iterations is: -9408.23505859375. Best ELBO loss value is: -10534.466796875.

x with CO2 means across time = tensor([[3.5768e+01, 2.0336e-01, 7.2395e-01, 6.5320e-02, 1.4435e-03],
        [3.5678e+01, 2.0901e-01, 7.2551e-01, 5.3849e-02, 1.4816e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [5.3290e+01, 1.5969e-01, 5.2912e-01, 9.3103e-02, 8.3373e-04],
         [5.2666e+01, 1.2854e-01, 7.5512e-01, 5.9816e-02, 8.9409e-04],
         ...,
         [3.6359e+01, 1.1807e-01, 8.4868e-01, 9.8428e-03, 1.8573e-03],
         [3.7066e+01, 1.4418e-01, 1.0487e+00, 2.4774e-02, 2.5213e-03],
         [3.7508e+01, 1.6684e-01, 8.8983e-01, 1.5735e-02, 2.6423e-03]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02, 6.8665e-04],
         [5.1196e+01, 4.8971e-03, 4.2790e-01, 5.6666e-02, 2.4371e-05],
         [4.9180e+01, 1.2349e-02, 4.2816e-01, 1.3061e-02, 6.3686e-05],
         ...,
  


Train Diffusion: 100%|██████████| 1500/1500 [20:03<00:00,  1.25it/s]


In [ ]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'AWB_ECA_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_ECA_C, SAWB_ECA_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB_ECA, get_CO2_AWB_ECA)


Train Diffusion:   0%|          | 0/1500 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000000204.209278. Best norm loss value is: 2042.0927734375.

x with CO2 means across time = tensor([[0.9142, 0.6837, 0.6677, 0.6208, 0.0037],
        [0.9173, 0.6829, 0.6602, 0.6142, 0.0036]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [7.6398e-01, 8.6637e-01, 7.1105e-01, 9.5738e-01, 3.9828e-03],
         [1.1127e+00, 4.9304e-01, 5.3768e-01, 5.7882e-01, 2.2650e-03],
         ...,
         [8.4045e-01, 6.0433e-01, 6.5992e-01, 4.3824e-01, 6.4206e-03],
         [9.8323e-01, 6.2523e-01, 4.8597e-01, 7.1255e-01, 5.2624e-03],
         [9.7902e-01, 9.6165e-01, 6.5284e-01, 7.9999e-01, 8.7898e-03]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [7.6398e-01, 8.9856e-01, 9.4289e-01, 4.6946e-01, 4.9687e-03],
         [8.0641e-01, 5.1886e-01, 6.7363e-01, 1.2792e+00, 2.7660e-03],
         ...,
         [8.5385e-01, 6.1986e-01, 4.4935e-01, 6.6087e


Train Diffusion:   1%|          | 10/1500 [00:08<20:49,  1.19it/s]

Moving average norm loss at 10 iterations is: 1944.9306030273438. Best norm loss value is: 1875.1636962890625.

x with CO2 means across time = tensor([[5.2645e+00, 3.6719e-01, 3.9715e-01, 4.0286e-01, 1.5966e-03],
        [5.7192e+00, 3.7323e-01, 3.9656e-01, 4.0293e-01, 1.5941e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [6.4901e-01, 1.3550e+00, 3.4522e-01, 4.2199e-01, 2.8868e-03],
         [4.0070e+00, 8.7708e-01, 3.9216e-01, 4.4924e-01, 2.6299e-03],
         ...,
         [9.8816e-01, 3.7636e-01, 4.6106e-01, 4.4741e-01, 3.4427e-03],
         [9.2898e+00, 3.6688e-01, 3.9763e-01, 3.5360e-01, 3.0231e-03],
         [6.0667e-01, 3.1147e-01, 3.4206e-01, 2.8671e-01, 2.3586e-03]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [6.4901e-01, 4.2298e+00, 3.9544e-01, 3.8751e-01, 4.9549e-03],
         [3.3494e-01, 1.5788e+00, 3.8191e-01, 3.2318e-01, 3.5335e-03],
         ...,
 


Train Diffusion:   1%|▏         | 20/1500 [00:16<22:21,  1.10it/s]

...


Train Diffusion:   7%|▋         | 100/1500 [01:20<18:03,  1.29it/s]

Moving average norm loss at 100 iterations is: 973.3418579101562. Best norm loss value is: 896.744384765625.

x with CO2 means across time = tensor([[3.3057e+01, 1.1385e-02, 3.4758e-03, 2.2810e-03, 2.5120e-06],
        [3.4238e+01, 1.5365e-02, 3.4995e-03, 2.2732e-03, 2.5153e-06]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [2.2528e+01, 6.3541e+00, 2.6054e-03, 2.2611e-03, 3.7499e-05],
         [4.9013e+01, 2.1855e-02, 2.4959e-03, 2.1927e-03, 1.0000e-06],
         ...,
         [2.5431e+01, 2.4990e-03, 2.5460e-03, 2.0199e-03, 1.0000e-06],
         [4.8081e+01, 2.6170e-03, 2.5612e-03, 1.7918e-03, 1.0000e-06],
         [1.9996e+01, 2.6695e-03, 2.2331e-03, 1.3729e-03, 1.0000e-06]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [1.1400e+01, 9.3417e+00, 2.6495e-03, 2.5918e-03, 3.9871e-05],
         [2.0369e+01, 2.3748e-02, 2.7632e-03, 2.6913e-03, 1.1091e-06],
         ...,
   


Train Diffusion:   7%|▋         | 110/1500 [01:28<18:31,  1.25it/s]

Moving average ELBO loss at 110 iterations is: 427288.2. Best ELBO loss value is: 272398.0625.

x with CO2 means across time = tensor([[3.4972e+01, 1.0840e-02, 3.3078e-03, 2.1983e-03, 2.4995e-06],
        [3.4860e+01, 8.7068e-03, 3.3227e-03, 2.2124e-03, 2.4974e-06]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [2.1501e+01, 6.8729e+00, 1.9336e-03, 2.0494e-03, 2.8122e-05],
         [4.4871e+01, 6.0148e-03, 1.9634e-03, 2.0043e-03, 1.0000e-06],
         ...,
         [3.0457e+01, 1.7188e-03, 3.0404e-03, 2.2809e-03, 1.0000e-06],
         [4.4659e+01, 1.6693e-03, 2.7108e-03, 1.2062e-03, 1.0000e-06],
         [2.1969e+01, 1.7853e-03, 2.6653e-03, 5.8594e-04, 1.0000e-06]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02, 1.1000e-03],
         [1.3560e+01, 5.2699e+00, 1.8946e-03, 2.2185e-03, 2.6527e-05],
         [2.6267e+01, 5.8708e-03, 2.1154e-03, 1.8121e-03, 1.0000e-06],
         ...,
         [4.4796e


Train Diffusion:   8%|▊         | 120/1500 [01:36<18:37,  1.23it/s]

In [ ]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'AWB_ECA_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_ECA_SS, SAWB_ECA_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB_ECA, get_CO2_AWB_ECA)